# CONCAT _0, _1 zarr to have only on file per type

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import matplotlib.pyplot as plt
import matplotlib.colors as cl

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm

from xgcm import Grid
from xhistogram.xarray import histogram
import warnings
warnings.filterwarnings("ignore")

import os
from glob import glob

import histlib.box as box
import histlib.aviso as aviso
import histlib.cstes as cstes
from histlib.cstes import labels, zarr_dir

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
zarr_dir_ok = zarr_dir+'_ok'

- i=0,1 :  jobs=6, cores = 26 
- i=2,3 :  jobs=1, cores = 26 
- i=4 :    jobs=1, cores = 10
- i=5 :    jobs=1, cores = 26
- i=6 :    jobs=4, cores = 26

In [3]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=10, processes=10, walltime='04:00:00')
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.62:8787/status,
Dashboard: http://10.148.1.62:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.62:57558,Workers: 0
Dashboard: http://10.148.1.62:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [14]:
for l in ["gps_Sentinel-3_A_2020"]:#labels:
    files = sorted(glob(os.path.join(zarr_dir, f'aviso_{l}_*.zarr')))
    for f in files :
        ds = xr.open_dataset(f).chunk({'obs':500})
        if ds.dims['aviso_time'] ==25 :#keep only five days of aviso (error for first drifter type, corrected after because too long)
            ds = ds.isel(aviso_time=slice(10, 15))
        if '_0.' in f :#overwrite
            ds.to_zarr(os.path.join(zarr_dir_ok,f'aviso_{l}.zarr'), mode = 'w')
        else : #append_dim
            ds.to_zarr(os.path.join(zarr_dir_ok,f'aviso_{l}.zarr'), append_dim='obs')

Frozen({'obs': 40000, 'box_y': 80, 'box_x': 120, 'aviso_time': 5, 'site_obs': 1489})
Frozen({'obs': 40000, 'box_y': 80, 'box_x': 120, 'aviso_time': 5, 'site_obs': 1489})
Frozen({'obs': 5718, 'box_y': 80, 'box_x': 120, 'aviso_time': 5, 'site_obs': 1489})


In [15]:
ds = xr.open_dataset(os.path.join(zarr_dir_ok,f'aviso_{l}.zarr')).chunk({'obs':500}).persist()

In [25]:
cluster.close()

In [ ]:
ds=xr